In [12]:
import tweepy
from os import getenv
from dotenv import load_dotenv
from neo4j import GraphDatabase
import pandas as pd
import time

load_dotenv()

True

In [13]:
consumer_key = getenv('consumer_key')
consumer_secret = getenv('consumer_secret')
access_token = getenv('access_token')
access_token_secret = getenv('access_token_secret')

In [3]:
df = pd.read_csv("output_follower.csv", index_col=[0])
df

,id,Name,Screen_Name,description,prifile_image,follower_id
0,1405886849441140739,藤田旭洋（Akihiro Fujita）,afujitabz,CEO of Bluezone inc. / 東大航空宇宙D2 / 未踏AD (2021) ...,https://pbs.twimg.com/profile_images/151408653...,"[1568798216069746689, 780896517859057665, 1052..."
1,1314113770285793280,いわもっち🌺マーケ部長,mocchi_iwa,マーケ部長やってます/ステキって言います/海が好き/沖縄好き/ダイバー兼サーファー兼webマ...,https://pbs.twimg.com/profile_images/156243996...,"[1567132777237123076, 1470039473282961411, 156..."
2,991303435617120257,sin・cos・カモノハシ,takadanobabauec,元天文 元DHK 元ⅡP4 元11クラ,https://pbs.twimg.com/profile_images/143929848...,[]
3,1543154097691705345,Uday kiran,Uday_Sugali,NaN,https://pbs.twimg.com/profile_images/154315416...,"[2504437416, 3155984131, 1331525617]"
4,1190436526175350784,ヨノ フリーランスエンジニア,yono_free,エンジニア歴9年。2児の父\nRails/React/Vue/TypeScript/gola...,https://pbs.twimg.com/profile_images/119043679...,"[38144769, 1470629774930616322, 15641015025702..."
...,...,...,...,...,...,...
1184,735982190,創英あるあるbot,soeiaru2,創英あるあるを思いついたらDMを気軽にください。 載せさせていただきます。 “あるある”と思...,https://pbs.twimg.com/profile_images/246550589...,"[1537394949356142592, 1499108026615476224, 142..."
1185,387729371,わっち,gracemeria,人生に、野遊びを,https://pbs.twimg.com/profile_images/326233015...,[]
1186,553560130,Heima Ueda,van_der_heima,青学 / LAMDAC2016 / 学生団体 参考書宅救便( @sankousho_SKU ...,https://pbs.twimg.com/profile_images/138833666...,[]
1187,311452958,角なし,EllenGrigia,ロンゾの面汚し,https://pbs.twimg.com/profile_images/599558314...,"[2374395042, 2939330144, 2582123257, 174270015..."


In [11]:
# 日本語を含める項目も加える
# strを加えて文字列に変換すると動く
main_name = "Y10iBobb"

driver = GraphDatabase.driver("neo4j+s://478aa850.databases.neo4j.io", auth=("neo4j", "xd_vaevy2Usjz5U3HMyxVCp4W1p0SJY7IjOg0pMi9hs"))
start = time.time()
# ここから先を検討中
def add_friend(tx, name, screen_name, twitter_id, Name, description, profile_image):
    tx.run("MERGE (a:Person {name: $name}) "
           "MERGE (friend:Person {name:$screen_name, twitter_id: $twitter_id, Name:$Name, description: $description, profile_image: $profile_image})-[:follow]->(a)",
           name=name, screen_name=screen_name, twitter_id=twitter_id, Name=Name, description=description, profile_image=profile_image)

with driver.session() as session:        
    for index, row in df.iterrows():
        twitter_id = row[0]
        Name = str(row[1])
        screen_name = row[2]
        description = str(row[3])
        profile_image= row[4]

        try:
            session.write_transaction(add_friend, main_name, screen_name, twitter_id, Name, description, profile_image)
        except:
            print("error_" + screen_name)

end = time.time()
driver.close()
print(end - start)

error_lol_st_02
error_technoprodesign
error_PayCareer_PR
error_fffff343452
error___3_141592__
error_naokiwifruit
error_yokoiwasaki6
error_hutami_siina_
error_botter_alpha
error_tanubitUT
error_kobobo_bo_ke
error_kokotaro66
error_0xis_1018
error_IPPO_furukawa
error_hiyokko_crypto
error_enji_tech
error_OxinasPicks
error_StudioOusia
error_shin_labs666
error_takasugi_mbsjk
error_pon3_dev
error_Ling16777398
error_takeya_ryou
error_digital_lab0
error_oh884U8
error_cats0830v
error_fujika_WA
error_comoselab
error_mczk9402
error_Nishika_koho
error_yuisudo24
error_yura_pppuhx
error_ei1333
error_aibiblio
error_goodengineer7
error_yakuma_en
error_hiropunk2077
error_GotGetCom
error_yuta_bluena
error_Soloware_
error_unpuy_tw
error_surasura
error_seikei_kin
error_ka_ttoo
error_hirock_e1983
error_ippo_012
error_shimabu_it
error_Samuraism
error_gabu_webdev
error_roma0218
error_yonkoma_uchu
error_hase_bitbank
error_hiroyasu01
error_AKT48000
error_ngsfield
error_taitai_244
error_keita_shimamaki
error_aki

In [6]:
df[df['Screen_Name'] == 'showgolaravel']

,id,Name,Screen_Name,description,prifile_image,follower_id
555,1036152500208857088,しょーご,showgolaravel,web制作やってます。目標はフルスタックWEBエンジニアになること。たまに関係ないツイートも...,https://pbs.twimg.com/profile_images/129171846...,"[1382759612856758273, 1522969420804595712, 363..."


In [8]:
df = df[555:]

In [18]:
df

,id,Name,Screen_Name,description,prifile_image,follower_id
555,1036152500208857088,しょーご,showgolaravel,web制作やってます。目標はフルスタックWEBエンジニアになること。たまに関係ないツイートも...,https://pbs.twimg.com/profile_images/129171846...,"[1382759612856758273, 1522969420804595712, 363..."
556,1357906029665603585,C山,Cyama_stat,京大M1 元情報系\n統計・データサイエンス、時々機械学習・深層学習 稀にマネジメント・コー...,https://pbs.twimg.com/profile_images/135790643...,"[1536456666, 1203731038570070016, 147579292656..."
557,1358027906581098497,heibei2020,heibei2020,エンジニアっぽい人,https://pbs.twimg.com/profile_images/135820546...,"[988542642458906624, 1444999236160946180, 6205..."
558,95898428,Taku,taku_sid,無所属 | 昨日より今日、今日より明日、日々変わり続ける事が大切\n|kaggle🥇0🥈2🥉...,https://pbs.twimg.com/profile_images/126740158...,"[4251510740, 1362665383279226882, 155474027010..."
559,1375084364585836544,hiroエンジニア,hiroengineerch1,hiroエンジニアと申します。\n\nブログ → https://t.co/SygCY6Ac...,https://pbs.twimg.com/profile_images/137508488...,"[1539056889979318272, 1407690261300342796, 151..."
...,...,...,...,...,...,...
1184,735982190,創英あるあるbot,soeiaru2,創英あるあるを思いついたらDMを気軽にください。 載せさせていただきます。 “あるある”と思...,https://pbs.twimg.com/profile_images/246550589...,"[1537394949356142592, 1499108026615476224, 142..."
1185,387729371,わっち,gracemeria,人生に、野遊びを,https://pbs.twimg.com/profile_images/326233015...,[]
1186,553560130,Heima Ueda,van_der_heima,青学 / LAMDAC2016 / 学生団体 参考書宅救便( @sankousho_SKU ...,https://pbs.twimg.com/profile_images/138833666...,[]
1187,311452958,角なし,EllenGrigia,ロンゾの面汚し,https://pbs.twimg.com/profile_images/599558314...,"[2374395042, 2939330144, 2582123257, 174270015..."


In [14]:
driver = GraphDatabase.driver("neo4j+s://478aa850.databases.neo4j.io", auth=("neo4j", "xd_vaevy2Usjz5U3HMyxVCp4W1p0SJY7IjOg0pMi9hs"))

In [19]:
# followerのfollowerを作成するところ

driver = GraphDatabase.driver("neo4j+s://478aa850.databases.neo4j.io", auth=("neo4j", "xd_vaevy2Usjz5U3HMyxVCp4W1p0SJY7IjOg0pMi9hs"))
start = time.time()

def add_friend_by_twitter_id(tx, name, twitter_id):
    tx.run("MERGE (a:Person {name: $name})"
           "MERGE (friend:Person {twitter_id: $twitter_id})-[:follow]->(a)",
           name=name, twitter_id=twitter_id)

with driver.session() as session:  
    for index, row in df.iterrows():
        main_name = row[2]
        print(main_name)
        followers_str = row[5].strip("[]")
        followers = followers_str.split(",")
        for follower in followers:
            try:
                follower_id = int(follower)
                session.write_transaction(add_friend_by_twitter_id, main_name, follower_id)
            except:
                print("error_", end=" ")
                print(follower_id)
            print(follower)

end = time.time()
driver.close()
print(end - start)

showgolaravel
1382759612856758273
 1522969420804595712
 363454595
 1555182407443443718
 1567786369715474433
 818362464236539905
 1132423882114125825
 87825328
 1263970819484479489
 1557431099873103874
 13989152
 1330348108983615489
 1382757589809721349
 1517476715353706496
 1541951325281619968
 1551044778136985600
 1016596888118673409
 1478610540490813441
 1514499584545865728
 1533301927291011072
 1493805324964626439
 3752458348
 1011981472561942528
 1259301259477016576
 6019712
 1190954314891612160
 1490480964514217984
 1496775297362178050
 136637617
 1477167672224002051
 1477435194437079040
 1251736976820105223
 1485850511652036608
 1686354678
 1372388972781998085
 950045801107005440
 1532628162
 1484902118826061827
 1447793540646330368
 3243361040
 1739721176
 100088141
 1297739744411938816
 40648435
 1324225730604015619
 1089449013676371969
 824085302264270850
 1250271021774426113
 1459059563945250816
 126638208
 14181938
 845275017432621058
 307170205
 955990427701723136
 14568158

 1486190500126552070
 1409456305434087424
 1273519897104924672
 111318772
 1479027895742119939
 1382053115537461248
 1884426416
 1032945393628106752
 1460052501580087296
 1458997558890864642
 1440370671842824198
 1354965506973863940
 1263254663681327104
 1122661421274128384
 1455799863992479745
 598212404
 1372788169351917571
 775484029638356992
 2992790190
 1322042169100623872
 1377199849527930885
 1428479222218203147
 886143527163371520
 1448313040009531393
 1446628398701371394
 1336112372218351616
 1350698938316144640
 1429558555472977923
 1279311075121364992
 1427791587162087428
 1408542525187493888
 1435635403793608708
 1115642848395661312
 1404795631810580486
 874222403471065089
 46114316
 3041196913
 1413766094850977796
 1132810158675914752
 1198426458613370881
 1264859004762681345
 986841366
 1411531719358894084
 1416232526515900417
 4335211878
 957461926115655685
 1180783143042740225
 94284111
 2983036626
 1415529085199994884
 1258027695192944646
 1337612721378889729
 14116240

Failed to write data to connection ResolvedIPv4Address(('34.66.78.163', 7687)) (IPv4Address(('34.66.78.163', 7687)))


 1347080330897883138
 1441319460468527110
 1526154128539283456
 1558375362262089728
 1555117732827627520
 1546299692455464960
 1536542957712871425
 1339751226481602560
 882471206993051648
 1082103608248721408
 1534191555158118400
 1458308607439941633
 1372382617585119236
 353691404
 1552937354385850369
 1549193793164914688
 1298471591940984833
 1550449770254045185
 1551774846291390464
 1552517046398619649
 1407475942054793220
 1549261941394182149
 1552433113632231426
 1129246101658193921
 1354716311129968645
 1519289705401307137
 1517096118957776896
 1520011590669836288
 1549257738118541312
 757218128963198976
 1546339911657455616
 193266175
 1501480154052567040
 1546728238910959616
 1547493957114142720
 1510799630535380992
 1547066361385746432
 1359456878003068932
 1543762868457582592
gintaro7749
394442109
 1340680635329904641
 1453910278978953218
 1551797496396976128
 1526921436463394816
 484010950
 1507257774249771008
 4709652433
 1323076870938660865
 1542111938846814209
 4508791753

 1151241016092925952
 1410501393073463296
 1410502574369906690
 1141566584219000833
 1343800692763369477
 1367383881314275330
 1365875491635220481
Dr_Matsushita
780201587721445377
 1401507927001747456
 1447126121670008838
 1333986708895846400
 1548473221799768065
 1547018688951267328
 4601748980
 1419264142708330501
 1535461642355343365
 1198906158917881856
 2693162863
 1434374500032811012
 1518689787267321856
 1565830991565819904
 1527823184073850880
 816254653302087680
 1438000020695908352
 1516374588988399621
 1489437904938373127
 1139664437361565696
 965785405139243009
 506537531
 1546772317220454401
 1522202335941779456
 1462232992785776640
 1528708710037884928
 1396433852319498242
 1429297687665414146
 1551843168123232256
 1360856514978664460
 1520904705651617793
 1259953877941547009
 1231817316389683200
 1404659237108666369
 1545020977025019904
 1140110883974049792
 1539552325664387072
 1506919526201716739
 1532718240853807106
 967012800106192897
 1127620689035386881
 1524753572

...

 1515615636315308035
 1303358256996253696
 1300094737228029956
 1555205346041688066
 1542147171600023555
 1367859636167081986
 1255442672589983746
 1541057577404231680
 1368511854163623937
 1445757294533246976
 4783195620
 908909658877829120
 1206424367388712962
 1377991943196614659
 1523524279190851584
 345292489
 1453265357167280131
 1102663285080383489
 1231570944985944066
 1309894324709027845
 1499695836892000257
 190236492
 1491591151560069120
 1515347642888298502
 2368430442
 1245279431209316354
 1506188525242183686
 1375069939984306184
 1255441999966228481
 779878553030537217
 1379394970667245574
 1362917787342004224
 958539942057074688
 1401420571355742214
 2854523580
 991284867550740481
 1389474084937895940
 1116618568299192321
 1301032618117328897
 1299916448618356736
 1254042839819796481
 1341399983384309765
 1659332251
 1443826733942071296
 232147203
 976017945145962496
 877538996271828992
 1631962040
 1414255100638367746
 977085239909498880
 1338840929357033473
 1147377771

Failed to write data to connection ResolvedIPv4Address(('34.66.78.163', 7687)) (IPv4Address(('34.66.78.163', 7687)))


 1019594407757033472
 1370772700797444098
 1382279578517872642
 1378202204041932800
 1333027779114278915
 1323505198237540352
 1240094668357918720
 2365631696
 1307316783666274306
 1213081043986604033
 1186557848265744385
 1162722521864523777
 1062184880212140032
 1518879810
 3021955292
 1121606338633592832
 1051501714619035649
 1117647094200672256
 1050063086726479872
 973484792586027008
 1076711928016203778
 757094707323219968
 2625737766
 2215845907
 1048212371418673152
 327300920
 1083553463911407616
 1082546799401852929
 957120103950180353
 1075994496611930112
 1921295210
 842696071755194368
 2496443126
 1540222212
 959055477807112192
 948211052998619136
 2649880016
 976316913532809217
 937329435463770113
 1063033834323664896
 1059340983471468544
 4732279855
 714384217719738368
 2405991313
 932513662664810497
 1050039688646414337
 2479915297
 968052609566236672
 1049098296692600832
 973478722861854720
 730167327237365761
 2827394466
 967530232457515008
 4853209754
 973079424441532

Failed to write data to connection ResolvedIPv4Address(('34.66.78.163', 7687)) (IPv4Address(('34.66.78.163', 7687)))


 1410506804
 879020112
 2364372600
 3227698290
 2457965839
 603642169
 627290175
 2380519338
 2573036640
 621926039
 626367488
 586634473
 3172313792
 73644299
 3149743892
 3042548954
 2282912552
 1048163120
 2703859645
 162707818
 2517405590
 2930564647
 2344858370
 545701749
 3107697308
 2458310545
 2679423948
 439964933
 2413955342
 161238628
 1025892163
 267600487
 3039625411
 2889745658
 1288050800
 2434951680
 1280468502
 2430702487
 113655688
 1461584420
 3010355964
 191402531
 2373321596
 342582295
 3013549369
 568910261
 2666889709
 2420204274
 1919274242
 2465256229
 2370601856
 2387191573
 2872470979
 817468801
 1227736814
 1338419707
 428093697
 179015409
 2382352926
 413769007
 1235164064
 2382711744
 2409185124
 109068334
 2890393812
 1302107005
 149451628
 2367181862
 831028592
 544074022
 398049945
 183588928
 100426318
 2365182780
 1540177314
 1042628419
soeiaru2
1537394949356142592
 1499108026615476224
 1424022834746060806
 747665882697043968
 2319151791
 543529333
 1